In [3]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#https://www.kaggle.com/aashita/nyt-comments

df = pd.read_csv('../file/ArticlesApril2018.csv')
# df.head()

headline = []
headline.extend(list(df.headline.values))
headline[:5]
# print(len(headline))

headline = [word for word in headline if word != 'Unknown']
# print(len(headline))

# headline[:5]

def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode('utf8').decode('ascii','ignore')
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


sequences = list()

for sentence in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

index_to_word = {}
for key, value in tokenizer.word_index.items():
    index_to_word[value] = key

print('빈도수 상위 582번 단어:{}'.format(index_to_word[582]))


3494
빈도수 상위 582번 단어:offer


In [4]:
max_len = max(len(l) for l in sequences)
print(max_len)

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

y = to_categorical(y, num_classes=vocab_size)
print(y[:3])
print(X[:3])

24
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200


2022-07-04 23:41:04.124048: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


244/244 - 3s - loss: 7.6450 - accuracy: 0.0276 - 3s/epoch - 13ms/step
Epoch 2/200
244/244 - 3s - loss: 7.1076 - accuracy: 0.0288 - 3s/epoch - 11ms/step
Epoch 3/200
244/244 - 3s - loss: 6.9748 - accuracy: 0.0364 - 3s/epoch - 11ms/step
Epoch 4/200
244/244 - 3s - loss: 6.8548 - accuracy: 0.0432 - 3s/epoch - 12ms/step
Epoch 5/200
244/244 - 3s - loss: 6.7141 - accuracy: 0.0436 - 3s/epoch - 11ms/step
Epoch 6/200
244/244 - 3s - loss: 6.5642 - accuracy: 0.0482 - 3s/epoch - 11ms/step
Epoch 7/200


In [ ]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        result = model.predict([encoded], verbose=0)
        result = np.argmax(result, axis=1)
        for word, index in tokenizer.word_index.items():
             if index == result:
                break
        
        current_word = current_word+ ' ' + word
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


print(sentence_generation(model, tokenizer, 'i', 10))
# i want to be rich and im not sorry with say 

i want to be rich and im not sorry with say


In [ ]:
sentence_generation(model, tokenizer, 'how', 10)

'how to make facebook more accountable to live in a bleached'

In [ ]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')

sentences = []

for sentence in f:
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.decode('ascii','ignore')
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()

total_data = ' '.join(sentences)

char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)



56


In [ ]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)
train_X = 'appl'
train_y = 'pple'

seq_length = 60

n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print ('샘플의 수 : {}'.format(n_samples))

NameError: name 'char_vocab' is not defined